### Thinking1：什么是近似最近邻查找，常用的方法有哪些


- NN（Nearest Neighbor Search),最近邻查找问题
- KNN(K-Nearest Neighbor Search),K近邻，查找离目标数据最近的前 K 个数据项(包括了半径和数量)
- ANN(Approximate Nearest Neighbor),近似最近邻检索，在牺牲可接受范围的精度的情况下提高检索的效率
- LSH, 局部敏感哈希是 ANN 的一种

### Thinking2：为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度


对于Ci与Cj，对应的行有三种可能：
- A类：两列的值都为1；
- B类：其中一列的值为0，另一列的值为1；
- C类：两列的值都为0.
C类行对于结果计算没有影响，可以删除


P(h(Ci)=h(Cj))=P(删掉C类后，第一行为A类)
=A类行的个数/所有行的个数=a/(a+b)


P(h(Ci)=h(Cj))= Jaccard(Ci,Cj)


用Ci，Cj的MinHash值相等的概率，对他们的Jaccard相似度进行估计


### Thinking3：SimHash在计算文档相似度的作用是怎样的？

- SimHash算法高效，适用于长文本， Google 将SimHash运用到网页的去重中去，具体来说：


互联网网页存在大量的重复内容网页，无论对于搜索引擎的网页去重和过滤、新闻小说等内容网站的内容反盗版和追踪，还是社交媒体等文本去重和聚类，都需要对网页或者文本进行去重和过滤。最简单的文本相似性计算方法可以利用空间向量模型，计算分词后的文本的特征向量的相似性，这种方法存在效率的严重弊端，无法针对海量的文本进行两两的相似性判断。模仿生物学指纹的特点，对每个文本构造一个指纹，来作为该文本的标识，从形式上来看指纹一般为固定长度较短的字符串，相同指纹的文本可以认为是相同文本。


### Thinking4：为什么YouTube采用期望观看时间作为评估指标


- 排序阶段采用观看时长作为学习目标而非点击率，由于常用的CTR指标对于视频搜索具有一定的欺骗性，比如诱导点击的标题党内容，用户点击后很快会停止观看，所以观看时长是一个更合适表示用户是否感兴趣的指标。所以作者提出采用期望观看时间作为评估指标。

### Action1：使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt ,针对某句话进行Query，查找Top-3相似的句子


In [1]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [ ]:
# 读取文件
f = open('./weibos.txt','r', encoding = 'utf-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉 \n 换行符号
sentences = re.split('[。 ！ ？]', text.replace('\n', ''))

# # 最后一行如果为空，则删除
if sentences[len(sentences) - 1] == '':
    sentences.pop()
sentences

In [35]:
# 将 item_text 进行分词
def get_item_str(item_text):
    item_str =  ''
    item = (pseg.cut(item_text))
    for i in list(item):
        # 去掉停用词
        if i.word not in list(stop):
            item_str += i.word
            # tfidf_vectorizer.fit_transform 的输入需要空格分隔的单词
            item_str += ''
    return item_str

# 对 item_str 创建 MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [36]:
# 设置停用词
stop = [line.strip() for line in open('./stopwords/cn_stopwords.txt', encoding = 'utf-8').readlines()]

In [37]:
# 得到停用次之后的 documents
documents = []
for item_text in sentences:
    # 将 item_text 进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)
    


In [39]:
# 创建 LSH Forest 及 MinHash 对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    # 得到 train_documents[i]的 MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)

In [40]:
# index 所有 key,以便可以进行检索
forest.index()

In [41]:
query = '国足昨晚-输给叙利亚赛后主帅里皮宣布辞职'
# 将 item_text 进行分词
item_str = get_item_str(query)
# 得到 item_str 的 MinHash 
minhash_query = get_minhash(item_str)

In [42]:
# 查询 forest 中与 m1 相识的 TOP-K个邻居
result = forest.query(minhash_query , 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ',''))
print('Top-3 邻居',result)

8 0.140625 比赛当晚太太西蒙内塔女士儿子小里皮现场台上观战
2 0.5625 ​国足输给叙利亚之后里皮辞职
44 1.0 国足昨晚-输给叙利亚赛后主帅里皮宣布辞职
Top-3 邻居 [8, 2, 44]


In [43]:
result

[8, 2, 44]